## выкачивание данных

In [3]:
import overpy
op = overpy.Overpass()

In [123]:
# http://gis.stackexchange.com/questions/178424/overpass-turbo-area-code-lookup
AREA = "3600060189"  # Russia

In [41]:
r = op.query('''
area({})->.searchArea;
(
    way
    ["name"]["highway"]
    (area.searchArea);
);
out;
'''.format(AREA))

In [47]:
import cPickle
with open('ways_ru.pcl', 'wb') as pcl:
    cPickle.dump(r, pcl)

## загрузка данных

In [1]:
import cPickle
with open('ways_ru.pcl', 'rb') as pcl:
    r = cPickle.load(pcl)

In [2]:
r.ways[1].tags

{'highway': 'tertiary',
 'name': u'\u041a\u043e\u043c\u0441\u043e\u043c\u043e\u043b\u044c\u0441\u043a\u0430\u044f \u0443\u043b\u0438\u0446\u0430',
 'name:en': "Komsomol'skaja street",
 'name:ru': u'\u041a\u043e\u043c\u0441\u043e\u043c\u043e\u043b\u044c\u0441\u043a\u0430\u044f \u0443\u043b\u0438\u0446\u0430',
 'postal_code': '183038',
 'surface': 'asphalt'}

In [3]:
import pandas as pd

df = pd.DataFrame(
    data=[w.tags for w in r.ways],
    index=[w.id for w in r.ways],
    columns=['name', 'highway', 'name:en', 'name:de']
)
df.head()

,name,highway,name:en,name:de
4078548,Шепетовская улица,residential,NaN,NaN
4391345,Комсомольская улица,tertiary,Komsomol'skaja street,NaN
4391346,проспект Ленина,primary,Lenina avenue,NaN
4391347,улица Полярные Зори,secondary,Poljarnye Zori street,NaN
4391349,улица Книповича,secondary,Knipovicha street,NaN


In [135]:
len(df)

659126

In [68]:
df.name.value_counts().head()

Советская улица      8672
улица Ленина         8444
Центральная улица    6983
Школьная улица       5531
Садовая улица        5492
Name: name, dtype: int64

In [4]:
streets_of = df[
    df.name.str.startswith(u'улица')
    | df.name.str.startswith(u'проспект')
]
streets_of.name.value_counts().head(15)

улица Ленина          8444
улица Мира            5095
улица Гагарина        3883
улица Кирова          3530
улица Пушкина         3012
улица Калинина        2843
улица Карла Маркса    2302
улица Горького        2209
улица Чапаева         2084
улица Победы          2019
улица Строителей      1725
улица Дзержинского    1645
улица Чкалова         1508
проспект Ленина       1485
улица Свердлова       1476
Name: name, dtype: int64

In [54]:
import pymorphy2
from functools32 import lru_cache

morph_analyzer = pymorphy2.MorphAnalyzer()

@lru_cache()
def street_to_person(s):
    inflected = s.split()[1:]
    morph = [morph_analyzer.parse(word) for word in inflected]
    normalized = []
    for m in morph:
        if not any('NOUN' in form.tag.grammemes for form in m):
            return None
        if not any(form.is_known for form in m):  # the word has not occurred on OpenCorpora, probably a rare surname
            normalized.append(m[0].normalized.word)
            continue
        for form in m:
            if form.is_known and form.tag.grammemes.intersection({'Name', 'Surn'}):
                normalized.append(m[0].normalized.word)
                break
        else:
            return None
    return ' '.join(normalized)

persons = streets_of.name.map(street_to_person).value_counts()
persons

ленин               9929
гагарин             4067
киров               3755
пушкин              3025
калинин             2922
карл маркс          2541
горький             2215
чапаев              2084
свердлов            1483
маяковский          1474
лермонтов           1466
мичурин             1430
фрунзе              1406
гоголь              1319
некрасов            1207
чехов               1117
энгельс             1076
островский          1071
суворов             1033
ломоносов           1002
крупский             987
шевченко             935
луначарский          923
максим горький       795
матросов             782
орджоникидзе         772
комаров              770
чернышевский         735
тургенев             718
володарский          712
                    ... 
сергей герасимов       1
алексей жомбин         1
имхаджиев              1
арон эрлихо            1
мардеев                1
семикин                1
биишева                1
бяков                  1
моге-киров             1


In [55]:
len(persons)

12818